# GPU CONFIGURATION

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Reduce TensorFlow logging
import tensorflow as tf
from tensorflow import keras

# Initialize TensorFlow
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        
# Check GPU availability
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Configure GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
        
print("====================================================================")     
print("||                     DRIVERS AND GPU INFO                       ||")       
print("====================================================================")      
print("")

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

# SYSTEM PARAMETERS

In [ ]:
import psutil

def get_ram_usage():
    return psutil.virtual_memory().used / (1024 ** 3) 

ram_usage_gb = get_ram_usage()
print(f"RAM Usage: {ram_usage_gb:.2f} GB")

# Get memory usage information
memory = psutil.virtual_memory()

# Total RAM in bytes
total_memory = memory.total

# RAM used in bytes
used_memory = memory.used

# RAM free in bytes
free_memory = memory.available

# RAM usage percentage
memory_percentage = memory.percent

print(f"Total Memory: {total_memory / (1024 ** 3)} GB")
print(f"Used Memory: {used_memory / (1024 ** 3)} GB")
print(f"Free Memory: {free_memory / (1024 ** 3)} GB")
print(f"Memory Usage: {memory_percentage}%")

print("")
print("====================================================================")
print("||                     SYSTEM INFO                                 ||")
print("====================================================================")
print("")

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

# VERIFICATION

In [ ]:
import os
HOME = os.getcwd()
print("Working at: ", HOME)

In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

# 1. Creation of the first model

In [ ]:
# Libraries for phase 1

from ultralytics import YOLO
from IPython.display import display, Image

# 1.1
import matplotlib.pyplot as plt
import numpy as np
import shutil

# 1.2
import os
from skimage import exposure
from PIL import Image

In [ ]:
model = YOLO("yolov8m.pt")

data_yaml_path = 'data/yolo_train_dataset/data.yaml'

results = model.train(
    data=data_yaml_path,
    epochs=150,
    imgsz=960,
    batch=16,
    name='TrueDentYOLOv8m',
    lr0=0.01, lrf=0.01, warmup_epochs=3, patience=50,
    device=0,  # 0 for GPU, -1 for CPU
)

metrics = model.val()
print(metrics)

In [ ]:
pt_path = '/home/mike/Desktop/codes/projects/AI_PRJ/TrueDent-AI/runs/detect/TrueDentYOLOv8m4/weights/best.pt'

# Carpeta destino y nombre final del .onnx exportado
export_dir = 'final_models'
export_name = 'TrueDent_v1.onnx'

# Crear carpeta si no existe
os.makedirs(export_dir, exist_ok=True)

# Cargar el modelo
model = YOLO(pt_path)

# Exportar a ONNX (se guarda al lado del .pt por defecto)
exported_model_path = model.export(format='onnx')

# Mover el archivo exportado a la ruta deseada
onnx_generated = os.path.splitext(pt_path)[0] + '.onnx'
onnx_target_path = os.path.join(export_dir, export_name)

# Verificamos que se haya creado
if os.path.exists(onnx_generated):
    shutil.move(onnx_generated, onnx_target_path)
    print(f"✅ Modelo exportado a: {onnx_target_path}")
else:
    print("❌ No se encontró el archivo .onnx generado.")
